# Convergence Testing of the Heat Equation
Check convergence using a high resolution surrogate solution

In [ ]:
using Plots; pyplot()
using Printf
using Heat1D
using LaTeXStrings
using LinearAlgebra
using Dierckx

In [ ]:
default(lw=2,markersize = 6,
    xtickfont=font(12), ytickfont=font(12), 
    guidefont=font(14), legendfont=font(12),titlefont=font(12))

## Convergence in Time

In [ ]:
a = 0;
b = 1;
nx = 99;
Δt_ref = 1.e-5;
tmax = 0.1;
x = LinRange(0,1,nx+2)[1:end-1];
# u0 = @. exp(-sin(2* π * x));
u0 = @. sin(2* π * x);

In [ ]:
problem_ref = FDHeatProblem(a, b, nx, Δt_ref, tmax, u0, PeriodicBC(), PeriodicBC(), BEuler());
t_trajectory, u_trajectory = solve_heat(problem_ref);
u_ref = u_trajectory[end];

In [ ]:
Δt_vals = 0.05 * 2. .^(-1:-1:-6);
@show Δt_vals

In [ ]:
err_vals = [];
for Δt in Δt_vals
    problem = FDHeatProblem(a,b, nx, Δt, tmax, u0, PeriodicBC(), PeriodicBC(), BEuler());
    t_trajectory, u_trajectory = solve_heat(problem);
    u = u_trajectory[end];
    err = norm(u .- u_ref,Inf);
    push!(err_vals, err);
end

In [ ]:
scatter(Δt_vals, err_vals, label=L"Error in $\|\|\bullet\|\|_\infty$", xscale=:log10, yscale=:log10, legend=:bottomright)
plot!(Δt_vals,1*Δt_vals, label=L"\propto \Delta t")
xlabel!(L"$\Delta t$")

## Convergence in Space

In [ ]:
a = 0;
b = 1;
nx_ref = 999;
Δt = 0.001;
tmax = 0.1;
x = LinRange(0,1,nx_ref+2)[1:end-1];
# u0 = @. exp(-sin(2* π * x));
u0 = @. sin(2* π * x);

In [ ]:
problem_ref = FDHeatProblem(a, b, nx_ref, Δt, tmax, u0, PeriodicBC(), PeriodicBC(), BEuler());
t_trajectory, u_trajectory = solve_heat(problem_ref);
u_ref = u_trajectory[end];
u_ref_spl = Spline1D([problem_ref.x; b], [u_ref; u_ref[1]], k=1);

In [ ]:
nx_vals = 10*2 .^(0:5) .-1;
Δx_vals = 1 ./(nx_vals.+1);
@show nx_vals

In [ ]:
err_vals = [];
for nx in nx_vals
    x = LinRange(0,1,nx+2)[1:end-1];
    u0 = @. sin(2* π * x);    
    problem = FDHeatProblem(a, b, nx, Δt, tmax, u0, PeriodicBC(), PeriodicBC(), BEuler());
    t_trajectory, u_trajectory = solve_heat(problem);
    u = u_trajectory[end];
    err = norm(u .- u_ref_spl.(problem.x),Inf);
    push!(err_vals, err);
end

In [ ]:
scatter(Δx_vals, err_vals, label=L"Error in $\|\|\bullet\|\|_\infty$", xscale=:log10, yscale=:log10, legend=:bottomright)
plot!(Δx_vals, Δx_vals.^2, label=L"\propto \Delta x^2")
xlabel!(L"$\Delta x$")